# הקדמה להסתברות וסטטיסטיקה
במחברת זו, נשחק עם כמה מהקונספטים שדנו בהם בעבר. רבים מהקונספטים של הסתברות וסטטיסטיקה מיוצגים היטב בספריות מרכזיות לעיבוד נתונים בפייתון, כגון `numpy` ו-`pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## משתנים אקראיים והתפלגויות
נתחיל בשרטוט דגימה של 30 ערכים מהתפלגות אחידה מ-0 עד 9. נחשב גם ממוצע ושונות.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

כדי להעריך ויזואלית כמה ערכים שונים יש בדגימה, אנחנו יכולים לשרטט את ה**היסטוגרמה**:


In [ ]:
plt.hist(sample)
plt.show()

## ניתוח נתונים אמיתיים

הממוצע והשונות הם חשובים מאוד כאשר מנתחים נתונים מהעולם האמיתי. בואו נטען את הנתונים על שחקני בייסבול מ-[SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> אנו משתמשים בחבילה בשם [**Pandas**](https://pandas.pydata.org/) כאן לניתוח נתונים. נדבר יותר על Pandas ועל עבודה עם נתונים בפייתון בהמשך הקורס הזה.

בואו נחשב ערכי ממוצע לגיל, גובה ומשקל:


In [ ]:
df[['Age','Height','Weight']].mean()

עכשיו נתמקד בגובה, ונחשב סטיית תקן ושונות:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

בנוסף לממוצע, יש הגיון לבחון את הערך החציוני והרבעונים. ניתן להמחיש אותם באמצעות **תרשים תיבתית**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

אנחנו יכולים גם להכין דיאגרמת קופסאות של תת-קבוצות ממערכת הנתונים שלנו, למשל, מקובצות לפי תפקיד השחקן.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **הערה**: דיאגרמה זו מציעה כי בממוצע, הגבהים של שחקני הבסיס הראשון גבוהים יותר מהגובה של שחקני הבסיס השני. בהמשך נלמד כיצד ניתן לבדוק השערה זו באופן פורמלי יותר, וכיצד להראות שהנתונים שלנו משמעותיים סטטיסטית כדי להוכיח זאת.  

הגיל, הגובה והמשקל הם כולם משתנים אקראיים רציפים. מה אתם חושבים לגבי התפלגותם? דרך טובה לגלות זאת היא על ידי הצגת היסטוגרמת הערכים: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## התפלגות נורמלית

בואו ליצור דגימה מלאכותית של משקלים שעוקבת אחרי התפלגות נורמלית עם אותו ממוצע ושונות כמו הנתונים האמיתיים שלנו:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

מכיוון שרוב הערכים בחיים האמיתיים מתפלגים באופן נורמלי, אין להשתמש במחולל מספרים אקראיים אחיד כדי ליצור נתוני דוגמה. הנה מה שקורה אם ננסה לייצר משקלים עם התפלגות אחידה (שהופקה על ידי `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## מרווחי אמון

עכשיו נחשב מרווחי אמון למשקלים והגדלים של שחקני בייסבול. נשתמש בקוד [מהדיון ב-stackoverflow הזה](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## בדיקת השערות

בואו נחקור תפקידים שונים במאגר השחקנים שלנו בבייסבול:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

בוא נבחן את ההשערה ששחקני ה-First Basemen גבוהים יותר משחקני ה-Second Basemen. הדרך הפשוטה ביותר לעשות זאת היא לבחון את מרווחי הביטחון:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

אנו יכולים לראות שהמרווחים אינם חופפים.

דרך סטטיסטית נכונה יותר להוכחת ההשערה היא להשתמש ב**מבחן t של סטודנט**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

שני הערכים שמוחזרים על ידי הפונקציה `ttest_ind` הם:
* ערך p ניתן להתייחס אליו כהסתברות ששתי ההתפלגויות הן בעלות ממוצע זהה. במקרה שלנו, הוא נמוך מאוד, מה שאומר שיש ראיות חזקות התומכות בכך שהבייסמנים הראשונים גבוהים יותר.
* ערך t הוא הערך הביניים של ההבדל הממוצע המנורמל שנעשה בו שימוש במבחן t, והוא מושווה לערך סף עבור ערך בטחון נתון.


## סימולציה של התפלגות נורמלית עם משפט הגבול המרכזי

הגנרטור הפסאודו-אקראי בפייתון מיועד לספק לנו התפלגות אחידה. אם נרצה ליצור גנרטור להתפלגות נורמלית, נוכל להשתמש במשפט הגבול המרכזי. כדי לקבל ערך שמתפלג באופן נורמלי פשוט נחשב את הממוצע של דגימה שנוצרה באופן אחיד.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## קורלציה ו-Evil Baseball Corp

קורלציה מאפשרת לנו למצוא קשרים בין רצפי נתונים. בדוגמת הצעצוע שלנו, נניח שיש תאגיד בייסבול מרושע שמשלם לשחקנים שלו בהתאם לגובהם - ככל שהשחקן גבוה יותר, הוא מקבל יותר כסף. נניח שיש שכר בסיס של 1000$, ובונוס נוסף מ-0$ עד 100$, תלוי בגובה. ניקח את השחקנים האמיתיים מ-MLB, ונחשב את המשכורות הדמיוניות שלהם:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

כעת נחשב את הקווריאנס והמתאם של הרצפים הללו. `np.cov` ייתן לנו את ה"matrix הקווריאנס", שהוא הרחבה של הקווריאנס למספר משתנים. האלמנט $M_{ij}$ במטריצת הקווריאנס $M$ הוא הקורלציה בין המשתנים $X_i$ ו-$X_j$, והערכים האלכסוניים $M_{ii}$ הם השונות של $X_{i}$. בדומה לכך, `np.corrcoef` תיתן לנו את מטריצת המתאם.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

קורלציה השווה ל-1 משמעותה שיש קשר **קוֹווי חָזָק** בין שני משתנים. ניתן לראות את הקשר הקווי באופן חזותי על ידי גרירת ערך אחד נגד השני:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

בוא נראה מה קורה אם הקשר לא יהיה ליניארי. נניח שהחברה שלנו החליטה להסתיר את התלות הליניארית הברורה בין גבהים למשכורות, והכניסה אלמנט של אי-ליניאריות לנוסחה, כמו `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

במקרה זה, המתאם קטן במעט, אך הוא עדיין גבוה למדי. עכשיו, כדי להפוך את הקשר לפחות ברור, אנו עשויים לרצות להוסיף קצת אקראיות נוספת על ידי הוספת משתנה אקראי לשכר. בואו נראה מה קורה:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> האם אתה יכול לנחש מדוע הנקודות מסתדרות לקווים אנכיים כאלו?

צפינו בקורלציה בין מושג מהונדס מלאכותית כמו שכר והמשתנה הנצפה *גובה*. בואו נראה גם אם שני המשתנים הנצפים, כמו גובה ומשקל, מתאמים גם כן:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

למרבה הצער, לא קיבלנו שום תוצאות - רק ערכי `nan` מוזרים. זה נובע מכך שחלק מהערכים בסדרה שלנו אינם מוגדרים, מיוצגים כ-`nan`, מה שגורם לתוצאה של הפעולה להיות גם היא בלתי מוגדרת. על ידי מבט במטריצה ניתן לראות ש`Weight` הוא העמודה הבעייתית, כי חושבה קורלציה עצמית בין ערכי `Height`.

> דוגמה זו מדגימה את חשיבות **הכנת הנתונים** ו**ניקויים**. ללא נתונים מתאימים לא נוכל לחשב כלום.

בואו נשתמש בשיטה `fillna` למילוי הערכים החסרים, ונחשב את הקורלציה:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

קיימת אכן מתאם, אך לא כזה חזק כמו בדוגמה המלאכותית שלנו. אכן, אם נסתכל על תרשים פיזור של ערך אחד כנגד השני, הקשר יהיה הרבה פחות ברור:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## סיכום

במחברת זו למדנו כיצד לבצע פעולות בסיסיות על נתונים כדי לחשב פונקציות סטטיסטיות. אנו כעת יודעים כיצד להשתמש במערכת מתמטית וסטטיסטית חזקה על מנת להוכיח השערות מסוימות, וכיצד לחשב אינטרוולים של ביטחון עבור משתנים אקראיים בהתבסס על מדגם נתונים.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**הצהרת אי-אחריות**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור שלו נחשב למקור הסמכותי. למידע קריטי מומלץ להשתמש בתרגום מקצועי של מתרגם אנושי. אנו לא נישא באחריות על הבנות שגויות או פרשנויות מוטעות הנובעות משימוש בתרגום זה.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
